In [1]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
df_ext = pd.read_csv('../data/raw/dating_app_behavior_dataset_extended1.csv')

In [3]:
df = df_ext.copy()

In [4]:
df['match_outcome'].unique()

array(['Mutual Match', 'Chat Ignored', 'Date Happened', 'No Action',
       'One-sided Like', 'Blocked', 'Relationship Formed', 'Catfished',
       'Ghosted', 'Instant Match'], dtype=object)

Positive: -> 1
- Mutual Match
- Date Happened
- Relationship Formed
- Instant Match

Negative:-> 0
- No Action
- One-sided Like
- Chat Ignored
- Ghosted
- Blocked
- Catfished

In [5]:
# change target to numerical
df['target_binary'] = df['match_outcome'].isin([
    'Mutual Match',
    'Date Happened',
    'Relationship Formed',
    'Instant Match'
]).astype(int)

In [6]:
df['target_binary'].value_counts(normalize=True)

target_binary
0    0.603
1    0.397
Name: proportion, dtype: float64

**Target**
match_outcome   → categorical target
target_binary   → binary target（0/1）

**Features**

1.Numerical -> StandardScaler

app_usage_time_min
swipe_right_ratio
likes_received
mutual_matches
profile_pics_count
bio_length
message_sent_count
emoji_usage_rate
last_active_hour
age
height_cm
weight_kg

2.Ordinal -> Ordinal Encoding

income_bracket
education_level
app_usage_time_label
swipe_right_label

3. Nominal -> One-Hot Encoding

gender
sexual_orientation
location_type
swipe_time_of_day
zodiac_sign
body_type
relationship_intent
interest_tags


In [7]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [8]:
numerical_cols

['app_usage_time_min',
 'swipe_right_ratio',
 'likes_received',
 'mutual_matches',
 'profile_pics_count',
 'bio_length',
 'message_sent_count',
 'emoji_usage_rate',
 'last_active_hour',
 'age',
 'height_cm',
 'weight_kg',
 'target_binary']

In [9]:
#VIF methond
X = df[numerical_cols].copy()

In [10]:
X = X.drop(columns=['target_binary'])

In [16]:
from statsmodels.tools.tools import add_constant

X_const = add_constant(X)

In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [14]:
vif_df = pd.DataFrame()
vif_df["feature"] = X_const.columns
vif_df["VIF"] = [
    variance_inflation_factor(X_const.values, i)
    for i in range(X_const.shape[1])
]

vif_df

,feature,VIF
0,const,198.174957
1,app_usage_time_min,1.000271
2,swipe_right_ratio,1.000121
3,likes_received,1.044663
4,mutual_matches,1.044624
5,profile_pics_count,1.000733
6,bio_length,1.000180
7,message_sent_count,1.000341
8,emoji_usage_rate,1.000182
9,last_active_hour,1.000451


In [17]:
vif_df[vif_df['VIF'] > 5].sort_values('VIF', ascending=False)

,feature,VIF
0,const,198.174957


Variance Inflation Factor (VIF) was calculated to assess multicollinearity among numerical features.
All features showed VIF values close to 1, with the exception of height and weight, which remained well below the commonly used threshold of 5.
This indicates a low level of multicollinearity, suggesting that the features provide largely independent information and that the logistic regression coefficients are stable and interpretable.

In [26]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

In [27]:
categorical_cols = [col for col in categorical_cols if col != 'match_outcome']

In [28]:
from scipy.stats import chi2_contingency

for col in categorical_cols:
    contingency = pd.crosstab(df[col], df['target_binary'])
    chi2, p, dof, expected = chi2_contingency(contingency)

    print(f"{col:25s} p-value = {p:.4e}")

gender                    p-value = 6.4590e-01
sexual_orientation        p-value = 4.8733e-01
location_type             p-value = 6.3417e-01
income_bracket            p-value = 7.2094e-01
education_level           p-value = 1.4193e-01
interest_tags             p-value = 3.5538e-01
app_usage_time_label      p-value = 9.2974e-01
swipe_right_label         p-value = 2.5878e-01
swipe_time_of_day         p-value = 6.5360e-02
zodiac_sign               p-value = 3.3349e-01
body_type                 p-value = 8.1485e-01
relationship_intent       p-value = 3.5005e-01


In [29]:
for col in categorical_cols:
    print(f"\n{col}")
    display(
        pd.crosstab(df[col], df['target_binary'], normalize='index')
    )


gender


target_binary,0,1
gender,,
Female,0.604127,0.395873
Genderfluid,0.604550,0.395450
Male,0.596002,0.403998
Non-binary,0.609371,0.390629
Prefer Not to Say,0.601200,0.398800
Transgender,0.602658,0.397342



sexual_orientation


target_binary,0,1
sexual_orientation,,
Asexual,0.603614,0.396386
Bisexual,0.593537,0.406463
Demisexual,0.606412,0.393588
Gay,0.597778,0.402222
Lesbian,0.600633,0.399367
Pansexual,0.601732,0.398268
Queer,0.607188,0.392812
Straight,0.612709,0.387291



location_type


...

target_binary,0,1
zodiac_sign,,
Aquarius,0.616663,0.383337
Aries,0.608426,0.391574
Cancer,0.605372,0.394628
Capricorn,0.593742,0.406258
Gemini,0.597475,0.402525
Leo,0.611820,0.388180
Libra,0.604673,0.395327
Pisces,0.594788,0.405212
Sagittarius,0.603159,0.396841



body_type


target_binary,0,1
body_type,,
Athletic,0.601513,0.398487
Average,0.604929,0.395071
Curvy,0.599168,0.400832
Muscular,0.607962,0.392038
Plus Size,0.605321,0.394679
Slim,0.599291,0.400709



relationship_intent


target_binary,0,1
relationship_intent,,
Casual Dating,0.609016,0.390984
Exploring,0.608385,0.391615
Friends Only,0.597073,0.402927
Hookups,0.601844,0.398156
Networking,0.595857,0.404143
Serious Relationship,0.605739,0.394261


Chi-square tests of independence were conducted to examine the relationship between categorical features and the binary match outcome.
Normalized contingency tables were additionally used to explore the direction and magnitude of observed differences.
The results indicate that most demographic and profile-related categorical variables, such as gender, sexual orientation, education level and relationship intent, do not show a statistically significant association with the likelihood of a mutual match.
This suggests that behavioral features may play a more important role than static personal attributes in predicting match outcomes on dating platforms.